# The instance variable

 Start by opening the result file `line.nc` as a postladim ParticleFile and consider the X-coordinate
 as an example of an instance variable. As in pandas and xarray, both dot notation pf.X and item notation
 pf["X"] can be used.

In [1]:
from postladim import ParticleFile
pf = ParticleFile("line.nc")
X = pf["X"]

In [2]:
# Printing the variable gives a short overview of the contents.
print(X)

<postladim.InstanceVariable>
num_times: 97, particle_instance: 97000
63.55 63.61 ... 120.73


In [3]:
# The underlying xarray DataArray is available as the da attribute

X.da

<xarray.DataArray 'X' (particle_instance: 97000)>
array([ 63.55    ,  63.61    ,  63.67    , ..., 123.93593 , 123.914566,
       120.73037 ], dtype=float32)
Dimensions without coordinates: particle_instance
Attributes:
    long_name:  particle X-coordinate

The most usual thing to do with an instance variable
to get the data at a certain timestep. This is returned
as an xarray DataArray.

In [4]:
# Get all values at timestep 4

X[3]

<xarray.DataArray 'X' (pid: 1000)>
array([ 63.488968,  63.550514,  63.61234 , ..., 123.28139 , 123.357155,
       123.434525], dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
  * pid      (pid) int32 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Attributes:
    long_name:  particle X-coordinate

Slicing also works and returns a new InstanceVariable.
Note that the step size in a slice must be one.

In [5]:
X[3:10]

<postladim.InstanceVariable>
num_times: 7, particle_instance: 7000
63.489 63.5505 ... 123.435

The data structure is optimized for information at a fixed time step.
It is also possible to follow a trajectory, a fixed particle id.
 This is done by the `sel` method.

In [6]:

X.sel(pid=7)




<xarray.DataArray (time: 97)>
array([63.97    , 63.954174, 63.93797 , 63.921387, 63.904423, 63.887077,
       63.869343, 63.851204, 63.83264 , 63.81364 , 63.794178, 63.77421 ,
       63.753716, 63.732643, 63.71094 , 63.688557, 63.66542 , 63.641457,
       63.616573, 63.590675, 63.563644, 63.535343, 63.505623, 63.47447 ,
       63.44186 , 63.40774 , 63.37237 , 63.33579 , 63.298004, 63.259018,
       63.218807, 63.177334, 63.13453 , 63.09031 , 63.04448 , 62.9969  ,
       62.947502, 62.896248, 62.843105, 62.788036, 62.731   , 62.67194 ,
       62.610798, 62.547516, 62.480476, 62.413597, 62.34673 , 62.279694,
       62.212296, 62.144424, 62.075985, 62.00664 , 61.936066, 61.86398 ,
       61.790157, 61.7156  , 61.64118 , 61.56707 , 61.49349 , 61.42508 ,
       61.362988, 61.307117, 61.25743 , 61.21391 , 61.17657 , 61.144608,
       61.119007, 61.10037 , 61.088745, 61.08419 , 61.086742, 61.09641 ,
       61.10916 , 61.10936 , 61.09326 , 61.061573, 61.014076, 60.948147,
       60.862072, 60.

The 'sel' and 'isel' method tries to follow xarray notation

In [7]:
# This is X[3]
print(X.sel(time="1989-05-24T21"))

# This is X[3, 7]
X.sel(time="1989-05-24T21", pid=7)

<xarray.DataArray 'X' (pid: 1000)>
array([ 63.488968,  63.550514,  63.61234 , ..., 123.28139 , 123.357155,
       123.434525], dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
  * pid      (pid) int32 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
Attributes:
    long_name:  particle X-coordinate


<xarray.DataArray 'X' ()>
array(63.921387, dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
    pid      int32 7
Attributes:
    long_name:  particle X-coordinate

Note that X[3, 7] is in general **not** equal to X[3][7]

 X[3][7] is the X value of the eighth particle at the fourth time step.
 X[3, 7] is the X value of the particle with particle identifier 7 at the fourth time
 step.
 If particles has been removed from the simulation, the eight particle may have a larger pid
 then 7. In our case, they are the same.

In [8]:
# These are equal
print(X[3, 7])
print(X.isel(time=3).sel(pid=7))
print(X.sel(pid=7).isel(time=3))

<xarray.DataArray 'X' ()>
array(63.921387, dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
    pid      int32 7
Attributes:
    long_name:  particle X-coordinate
<xarray.DataArray 'X' ()>
array(63.921387, dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
    pid      int32 7
Attributes:
    long_name:  particle X-coordinate
<xarray.DataArray ()>
array(63.921387, dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
    pid      int64 7


 It is interesting to look at the timings. The two first are equal
 (same implementation (SJEKK). The last take 100 times more.
 The reason is that it first has to select values for pid=7 at all time steps, a slow operation.

In [12]:
%timeit -o X[3, 7]
%timeit -o X.isel(time=3).sel(pid=7)
%timeit -o X.sel(pid=7).isel(time=3);

1.41 ms ± 60.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.39 ms ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
144 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 144 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [10]:
# These are equal.
# Note that X.isel(pid=7) is not defined
print(X[3][7])
print(X.isel(time=3).isel(pid=7))

<xarray.DataArray 'X' ()>
array(63.921387, dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
    pid      int32 7
Attributes:
    long_name:  particle X-coordinate
<xarray.DataArray 'X' ()>
array(63.921387, dtype=float32)
Coordinates:
    time     datetime64[ns] 1989-05-24T21:00:00
    pid      int32 7
Attributes:
    long_name:  particle X-coordinate


The InstanceVariable can be returned as a full 2D DataArray.
This makes it equally simple to extract trajectories as time snapshots.

Note that for a long simulation with continuous particle release and
extensive particle death, this array may become very large.

In [11]:
X.full()

<xarray.DataArray (time: 97, pid: 1000)>
array([[ 63.54999924,  63.61000061,  63.66999817, ..., 123.33000183,
        123.38999939, 123.44999695],
       [ 63.52970123,  63.59034348,  63.6509819 , ..., 123.31063843,
        123.37738037, 123.4442215 ],
       [ 63.50925827,  63.57050705,  63.63175583, ..., 123.29421234,
        123.3663559 , 123.43903351],
       ...,
       [ 59.93954849,  59.89281082,  59.86213303, ..., 123.93592834,
        123.91456604, 120.77573395],
       [ 59.91397476,  59.8729248 ,  59.84598541, ..., 123.93592834,
        123.91456604, 120.7530899 ],
       [ 59.89163208,  59.8551445 ,  59.8312149 , ..., 123.93592834,
        123.91456604, 120.73036957]])
Coordinates:
  * time     (time) datetime64[ns] 1989-05-24T12:00:00 ... 1989-06-05T12:00:00
  * pid      (pid) int32 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999